# Multi Agent Reinforcement Learning Information Extraction

## Agents
-> **Structured Information Retrieval**: A model that uses a structured approach to extract information from unstructured data. (Numind)
- **Name Entity Recognition**: A model that identifies and classifies named entities in text into predefined categories
  (e.g., person names, organizations, locations). (Spacy)
- **Relation Extraction**: A model that identifies and classifies relationships between entities in text. (Spacy)

## Reinforcement Learning??
-> **Agent Relationship**: The relationship between the agent and the environment, where the agent interacts with the environment to learn and improve its performance.
- Cooperative setting - All agents share the aligning goal.
- Adversarial setting - Agents have dichotomous goals.
- Mixed setting - Agents have varying goals.


## Reward Functions
- **Syntax Reward**: A reward function that evaluates the syntactic correctness of the generated text.

In [ ]:
# Recieve user unstructured data input


# Start Agent loop (https://ampcode.com/how-to-build-an-agent)


# NER - Named Entity Recognition probably using spacy


# Relation Extraction - using OpenIE or similar

# Generate prompt for LLM to generate cypher query
# using LangChain or similar

# Generate prompt for visualizing the graph with Neo4j

# Finalize the graph and display it